In [1]:
import os
import pandas as pd

In [2]:
output_folder_electionresult = "election_results"
output_folder_partiesresult = "winners.csv"

parliament_df = pd.read_csv("election_analysis/electoral_district.csv")

# Create an empty list to store the winners
winners = []

In [3]:
import os
import pandas as pd

output_folder_electionresult = "election_results"
output_folder_partiesresult = "winners.csv"
parliament_df = pd.read_csv("election_analysis/electoral_district.csv")

# Define the list of election prefixes
election_prefixes = ["ge14", "ge15"]

# Create an empty list to store the winners
winners = []

# Iterate through the election prefixes
for prefix in election_prefixes:
    # Iterate through the constituency folders
    for folder in os.listdir(output_folder_electionresult):
        constituency_folder = os.path.join(output_folder_electionresult, folder)
        file_prefix = prefix + "_"

        # Iterate through the CSV files for the current election
        for file in os.listdir(constituency_folder):
            if file.startswith(file_prefix):
                file_path = os.path.join(constituency_folder, file)

                # Read the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Rename the parties accordingly                             
                if prefix == "ge14":
                    df["Party"] = df["Party"].replace("Pan-Malaysian Islamic Party", "PAS")
                    df["Party"] = df["Party"].replace("Homeland Solidarity Party", "STAR")
                    df["Party"] = df["Party"].replace("Sabah Heritage Party", "WARISAN")
                    df["Party"] = df["Party"].replace("DAP", "PH")
                elif prefix == "ge15":
                    df["Party"] = df["Party"].replace("DAP", "PH")
                    df["Party"] = df["Party"].replace("Pan-Malaysian Islamic Party", "PN")
                    df["Party"] = df["Party"].replace("Sabah Heritage Party", "WARISAN")
                
                # Determine the winner based on the party with the highest vote count
                winner = df.loc[df["Vote Count"].idxmax(), "Party"]
                highest_vote = df["Vote Count"].max()
                second_highest_vote = df[df["Vote Count"] != highest_vote]["Vote Count"].max()

                # Calculate the percentage of vote
                total_votes = df["Vote Count"].sum()
                df["Percentage of Vote"] = round((df["Vote Count"] / total_votes * 100), 2)

                # Calculate the majority
                majority = highest_vote - second_highest_vote
                percentage_majority = round((majority / total_votes * 100), 2)

                # Append the winner and other data to the list
                winners.append({
                    "Election": prefix,
                    "Parliament Code": folder,
                    "Federal Constituency": parliament_df.loc[parliament_df["Parliament Code"] == folder, "Federal Constituency"].values[0],
                    "Alliance": winner,
                    "Majority": majority,
                    "Percentage of Vote": df.loc[df["Party"] == winner, "Percentage of Vote"].values[0] if len(df.loc[df["Party"] == winner]) > 0 else None,
                    "Total Voters in Constituency": total_votes,
                    "Margin-of-Votes": percentage_majority
                })

# Create a DataFrame from the winners list
winners_df = pd.DataFrame(winners)

# Reorder the columns
winners_df = winners_df[["Election", "Parliament Code", "Federal Constituency", "Alliance", "Majority", "Percentage of Vote", "Total Voters in Constituency", "Margin-of-Votes"]]

# Save the winners DataFrame as a CSV file
#winners_df.to_csv(output_folder_partiesresult, index=False)



In [4]:
# Create a new column 'Classification' in the winners_df DataFrame
winners_df["Classification"] = ""

# Iterate through each row in the DataFrame
for index, row in winners_df.iterrows():
    percentage_majority = row["Margin-of-Votes"]

    # Assign the classification based on the percentage majority
    if 0.00 <= percentage_majority <= 5.99:
        winners_df.at[index, "Classification"] = "Marginal"
    elif 6.00 <= percentage_majority <= 9.99:
        winners_df.at[index, "Classification"] = "Fairly Safe"
    elif 10.00 <= percentage_majority <= 18.99:
        winners_df.at[index, "Classification"] = "Safe"
    elif percentage_majority >= 19.00:
        winners_df.at[index, "Classification"] = "Very Safe"

# Print the updated DataFrame
winners_df.tail()
#winners_df.to_csv("election_analysis/parliament_margin.csv", index=False)

,Election,Parliament Code,Federal Constituency,Alliance,Majority,Percentage of Vote,Total Voters in Constituency,Margin-of-Votes,Classification
438,ge15,p218,Sibuti,GPS,11745,65.31,33916,34.63,Very Safe
439,ge15,p219,Miri,PH,6159,50.61,78148,7.88,Fairly Safe
440,ge15,p220,Baram,GPS,7339,61.78,29783,24.64,Very Safe
441,ge15,p221,Limbang,GPS,9998,75.25,19796,50.51,Very Safe
442,ge15,p222,Lawas,GPS,5677,62.40,18208,31.18,Very Safe


In [5]:
import os

# Create a directory to store the output files
output_folder = "parties_results"
os.makedirs(output_folder, exist_ok=True)

# Iterate through unique elections in the winners dataframe
for election in winners_df["Election"].unique():
    # Filter the winners for the current election
    election_winners = winners_df[winners_df["Election"] == election]
    
    # Define the output file path
    output_path = os.path.join(output_folder, f"{election}_partiesresults.csv")
    
    # Save the election winners to CSV
    election_winners.to_csv(output_path, index=False)


In [6]:
# Create a directory to store the output files
output_folder = "parties_results"
os.makedirs(output_folder, exist_ok=True)

# Define the alliances and minor parties for each election
elections = [
    {
        "name": "ge14",
        "alliances": ["PAS", "PH", "BN"],
        "minor_parties": ["Independent", "WARISAN", "STAR"]
    },
    {
        "name": "ge15",
        "alliances": ["PN", "PH", "BN", "GPS", "GRS"],
        "minor_parties": ["Independent", "WARISAN", "MUDA", "PBM", "KDM"]
    }
]

# Iterate through the elections
for election in elections:
    election_name = election["name"]
    alliances = election["alliances"]
    minor_parties = election["minor_parties"]

    # Filter the winners DataFrame for the current election
    election_df = winners_df[winners_df["Election"] == election_name]

    # Iterate through the alliances
    for alliance in alliances:
        # Filter the DataFrame for the current alliance
        alliance_df = election_df[election_df["Alliance"].str.contains(alliance)].copy()

        # Remove duplicate rows
        alliance_df = alliance_df.drop_duplicates()

        # Select the desired columns
        alliance_df = alliance_df[["Federal Constituency", "Parliament Code", "Margin-of-Votes", "Classification"]]

        # Sort the DataFrame by increasing percentage of majority
        alliance_df = alliance_df.sort_values(by="Margin-of-Votes")

        # Save the DataFrame to a CSV file
        alliance_df.to_csv(f"{output_folder}/{election_name}_{alliance.lower()}_seat.csv", index=False)

    # Process the minor parties individually
    minor_df = pd.DataFrame(columns=["Party", "Federal Constituency", "Parliament Code", "Margin-of-Votes", "Classification"])

    for party in minor_parties:
        # Filter the DataFrame for the current party
        party_df = election_df[election_df["Alliance"] == party].copy()

        # Remove duplicate rows
        party_df = party_df.drop_duplicates()

        # Select the desired columns
        party_df = party_df[["Federal Constituency", "Parliament Code", "Margin-of-Votes", "Classification"]]

        # Add the "Party" column
        party_df["Party"] = party

        # Sort the DataFrame by increasing percentage of majority
        party_df = party_df.sort_values(by="Margin-of-Votes")

        # Append the party DataFrame to the minor DataFrame
        minor_df = minor_df.append(party_df, ignore_index=False)

    # Save the minor DataFrame to a CSV file
    minor_df.to_csv(f"{output_folder}/{election_name}_minor_seat.csv", index=False)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9576\1583100448.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  minor_df = minor_df.append(party_df, ignore_index=False)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9576\1583100448.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  minor_df = minor_df.append(party_df, ignore_index=False)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9576\1583100448.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  minor_df = minor_df.append(party_df, ignore_index=False)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9576\1583100448.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  minor_df = minor_